## 5.4 損失関数

### 5.4.1 二乗和誤差

In [1]:
import numpy as np

def square_sum(y, t): # 出力値と正解値を引数に取る
    return 1.0 / 2.0 * np.sum(np.square(y - t))

### 5.4.2 交差エントロピー誤差

In [2]:
import numpy as np

def cross_entropy(y, t): # 出力値と正解値を引数に取る
    return - np.sum(t * np.log(y + 1e-7)) # 1e-7は無限小に発散することを防いでいる

## 5.5 勾配降下法